In [1]:
from primeqa.components.retriever.dense import ColBERTRetriever
from primeqa.components.retriever.dense import DPRRetriever
from primeqa.components.reader.prompt import PromptGPTReader
from primeqa.pipelines.qa_pipeline import QAPipeline
import json

{"time":"2023-02-13 11:44:07,069", "name": "faiss.loader", "level": "INFO", "message": "Loading faiss with AVX2 support."}
{"time":"2023-02-13 11:44:07,385", "name": "faiss.loader", "level": "INFO", "message": "Successfully loaded faiss with AVX2 support."}


In [ ]:
# setup ColBERT index
index_root = "/dccstor/mabornea1/kilt-wikipedia-test/colbert_ir/kilt_wikipedia_eli5_dev_exp/indexes/"
index_name = "kilt_wikipedia_eli5_dev_indname"
collection = "/dccstor/mabornea1/kilt-wikipedia-test/passages/kilt_knowledgesource_eli5_dev.tsv"


retriever = ColBERTRetriever(index_root = index_root, 
                                     index_name = index_name, 
                                     collection = collection, 
                                     max_num_documents = 3)
retriever.load()

In [2]:
#checkpoint = "/dccstor/colbert-ir/franzm/experiments/jan3_7_10/qry_encoder"
checkpoint = "/dccstor/colbert-ir/franzm/experiments/oct2_18_10/qry_encoder"
index_root = "/dccstor/colbert-ir/franzm/experiments/"
index_name = "oct2_18_20"
collection = "/dccstor/avi7/neural_ir/colbert/data/psgs_w100.tsv"

retriever = DPRRetriever(index_root = index_root, 
                                     index_name = index_name,  
                                     checkpoint = checkpoint,
                                     collection = collection,
                                     max_num_documents = 3)

retriever.load()

{"time":"2023-02-13 11:44:16,054", "name": "primeqa.ir.dense.dpr_top.torch_util.hypers_base", "level": "INFO", "message": "world_rank 0 cuda_is_available True cuda_device_cnt 1 on cccxc564, CUDA_VISIBLE_DEVICES = 0"}
{"time":"2023-02-13 11:45:05,574", "name": "primeqa.ir.dense.dpr_top.dpr.searcher", "level": "INFO", "message": "Using sharded faiss, reading shards from /dccstor/colbert-ir/franzm/experiments/oct2_18_20"}
{"time":"2023-02-13 11:45:05,575", "name": "primeqa.ir.dense.dpr_top.dpr.searcher", "level": "INFO", "message": "Reading passages_1_of_40.json.gz.records"}
{"time":"2023-02-13 11:45:09,898", "name": "primeqa.ir.dense.dpr_top.dpr.searcher", "level": "INFO", "message": "Reading passages_15_of_40.json.gz.records"}
{"time":"2023-02-13 11:45:13,245", "name": "primeqa.ir.dense.dpr_top.dpr.searcher", "level": "INFO", "message": "Reading passages_4_of_40.json.gz.records"}
{"time":"2023-02-13 11:45:15,941", "name": "primeqa.ir.dense.dpr_top.dpr.searcher", "level": "INFO", "messag

We provide three examples of loading models to generate answers from the prompt: GPT, Flan-T5-xxl from HuggingFace, and Flan-T5-xxl from BAM services

In [3]:
# setup a Prompt Reader: here we use a GPT reader
reader = PromptGPTReader(api_key='')
reader.load()

In [4]:
# setup the pipeline
pipeline = QAPipeline(retriever, reader)

21015325it [01:00, 347195.41it/s]


In [5]:
# start asking questions
questions = ["Does smoking marijuana impair driving ?"]
answers = pipeline.run(questions, use_retriever=False)
print(json.dumps(answers, indent=4))

[
    {
        "example_id": 0,
        "text": "\n\nAnswer: Yes, smoking marijuana can impair driving. Studies have shown that smoking marijuana can slow reaction time, impair judgment, coordination and concentration, and decrease the ability to make quick decisions necessary for safe driving."
    }
]


In [6]:
# start asking questions
questions = ["Does smoking marijuana impair driving ?"]
prompt_prefix = "Answer the following question after looking at the text."
answers = pipeline.run(questions, prefix=prompt_prefix, use_retriever=True)
print(json.dumps(answers, indent=4))

[
    {
        "example_id": 0,
        "text": "\n\nNo, the text does not mention anything about smoking marijuana impairing driving.",
        "passages": [
            "\"tobacco companies are viewed and dismissed as \"\"deceitful\"\", i.e. as being driven by (conscious) bad intention. During the debates over the Washington, DC, smoke-free law, city council member Carol Schwartz proposed legislation that would have enacted either a substantial tax credit for businesses that chose to voluntarily restrict smoking or a quadrupling of the annual business license fee for bars, restaurants and clubs that wished to allow smoking. Additionally, locations allowing smoking would have been required to install specified high-performance ventilation systems. Critics of smoke-free laws have suggested that ventilation is a means of reducing the harmful effects of second-hand\"",
            "\"smoking ban ordinances that their legislature found unacceptable. In Wauwatosa, Wisconsin, three restaur

In [7]:
# start asking questions
questions = ["What causes the trail behind jets at high altitude? ?"]
answers = pipeline.run(questions, use_retriever=False)
print(json.dumps(answers, indent=4))

[
    {
        "example_id": 0,
        "text": "\n\nAnswer: The trail behind jets at high altitude is called a contrail, which is short for \"condensation trail\". Contrails are composed of water vapor that is released from the jet's engines, which then quickly freezes in the cold air at high altitudes. The resulting ice crystals form a visible plume-like trail behind the jet."
    }
]


In [8]:
# start asking questions
questions = ["What causes the trail behind jets at high altitude? ?"]
prompt_prefix = "Answer the following question after looking at the text."
answers = pipeline.run(questions, prefix=prompt_prefix, use_retriever=True)
print(json.dumps(answers, indent=4))

[
    {
        "example_id": 0,
        "text": "\n\nThe trail behind jets at high altitude is caused by their updrafts and outflow boundaries.",
        "passages": [
            "\"before the traveller is fully adjusted to the new time zone; a recovery period of one day per time zone crossed is a suggested guideline. Jet lag is especially an issue for airline pilots, aircraft crew, and frequent travellers. Airlines have regulations aimed at combating pilot fatigue caused by jet lag. The term \"\"jet lag\"\" is used because before the arrival of passenger jet aircraft, it was uncommon to travel far and fast enough to cause desynchronosis. Travel by propeller-driven aircraft, by ship, or by train was slower and of more limited distance than jet flights, and thus did not\"",
            "\"Texas area and many people associated with DFW and American Airlines were reluctant to put any jobs at risk, especially when considering the chronic financial difficulties that modern airlines, other

In [ ]:
#setup a FlanT5PromptReader: note this loads a Flan T5 xxl model
from primeqa.components.reader.prompt import PromptFLANT5Reader
reader = PromptFLANT5Reader()
reader.load()

In [ ]:
# setup the pipeline
pipeline = QAPipeline(retriever, reader)

In [17]:
# start asking questions
questions = ["Does smoking marijuana impair driving ?"]
prompt = "Answer the following question after looking at the text."
answers = pipeline.run(questions,prompt)
print(json.dumps(answers, indent=4))

[Feb 05, 15:18:28] #> XMLR QueryTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
[Feb 05, 15:18:28] #> Input: $ Does smoking marijuana impair driving ?, 		 True, 		 None
[Feb 05, 15:18:28] #> Output IDs: torch.Size([32]), tensor([     0,   9748, 101790, 200166, 227071,    566, 109637, 132442,    705,
             2,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1])
[Feb 05, 15:18:28] #> Output Mask: torch.Size([32]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
[Feb 05, 15:18:29] #>>>> colbert query ==
[Feb 05, 15:18:29] #>>>>> input_ids: torch.Size([32]), tensor([     0,   9748, 101790, 200166, 227071,    566, 109637, 132442,    705,
             2,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,  

100%|██████████| 1/1 [00:00<00:00, 66.32it/s]
/dccstor/srosent1/miniconda3/envs/primeqa4.24/lib/python3.9/site-packages/transformers/generation_utils.py:1359: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[
    {
        "example_id": 0,
        "text": [
            "\"[a]ny amount of marijuana consumption puts you at risk of driving impaired.\"\""
        ],
        "passages": [
            "\"In the US state of Colorado, the state government indicates that \"\"[a]ny amount of marijuana consumption puts you at risk of driving impaired.\"\" Colorado law states that \"\"drivers with five nanograms of active tetrahydrocannabinol (THC) in their whole blood can be prosecuted for driving under the influence (DUI). However, no matter the level of THC, law enforcement officers base arrests on observed impairment.\"\" In Colorado, if consumption of marijuana is impairing your ability to drive, \"\"it is illegal for you to be driving, even if that substance is prescribed [by a doctor] or legally acquired.\"\" Section: Other drugs.:Prescription medications.\"",
            "\"Section: Legal standards.:Americas.:United States.:State.:Georgia. Georgia Statute \u00a7 40-6-391 makes it to drive whi

In [18]:
# start asking questions
questions = ["What causes the trail behind jets at high altitude? ?"]
prompt = "Answer the following question after looking at the text."
answers = pipeline.run(questions,prompt)
print(json.dumps(answers, indent=4))

100%|██████████| 1/1 [00:00<00:00, 77.71it/s]


[
    {
        "example_id": 0,
        "text": [
            "condensation trails"
        ],
        "passages": [
            "\"Aircraft flying at high altitude form condensation trails or contrails in the exhaust plume of their engines. While in the Troposphere these have very little climatic impact. However, jet aircraft cruising in the Stratosphere do create an impact from their contrails, although the extent of the damage to the environment is as yet unknown. Contrails can also trigger the formation of high-altitude Cirrus cloud thus creating a greater climatic effect. A 2015 study found that artificial cloudiness caused by contrail \"\"outbreaks\"\" reduce the difference between daytime and nighttime temperatures. The former are decreased and the latter are increased, in comparison\"",
            "\"Contrails, or condensation trails, are \"\"streaks of condensed water vapor created in the air by an airplane or rocket at high altitudes.\"\" Fossil fuel combustion (as in pisto

In [19]:
#setup a FlanT5PromptReader using the bam service (https://bam.res.ibm.com/): note this loads a Flan T5 xxl model
from primeqa.components.reader.prompt import PromptFLANT5Reader
reader = PromptFLANT5Reader(use_bam=True, api_key='API KEY HERE')
reader.load()

In [20]:
# setup the pipeline
pipeline = QAPipeline(retriever, reader)

14635it [00:00, 283158.38it/s]


In [21]:
# start asking questions
questions = ["Does smoking marijuana impair driving ?"]
prompt = "Answer the following question after looking at the text."
answers = pipeline.run(questions,prompt)
print(json.dumps(answers, indent=4))

100%|██████████| 1/1 [00:00<00:00, 36.33it/s]
/dccstor/srosent1/miniconda3/envs/primeqa4.24/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'bam-api.res.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


[
    {
        "example_id": 0,
        "text": "\"[a]ny amount of marijuana consumption puts you at risk of driving impaired.\"\" Colorado law states that \"\"drivers with five nanograms of active tetrahydrocannabinol (THC) in their whole blood can be prosecuted for driving under the influence (DUI). However, no matter the level of THC, law enforcement officers base arrests on observed impairment.\"\" In Colorado, if consumption of marijuana is impairing your ability to drive, \"\"it is illegal for you to be driving, even if that substance is prescribed [by a doctor] or legally acquired.\"\" Section: Other drugs.:Prescription medications.\", \"Section: Legal standards.:Americas.:United States.:State.:Georgia. Georgia Statute  40-6-391 makes it to drive while there is \"\"\"\"any amount\"\" of marijuana . . . present in the person's blood or urine, or both, \"\"including the metabolites and derivatives of each or both\"\"\"\" Subsection (b) of the statute provides that being legally e

In [22]:
# start asking questions
questions = ["What causes the trail behind jets at high altitude? ?"]
prompt = "Answer the following question after looking at the text."
answers = pipeline.run(questions,prompt)
print(json.dumps(answers, indent=4))

100%|██████████| 1/1 [00:00<00:00, 58.55it/s]
/dccstor/srosent1/miniconda3/envs/primeqa4.24/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'bam-api.res.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


[
    {
        "example_id": 0,
        "text": "condensation trails or contrails in the exhaust plume of their engines. While in the Troposphere these have very little climatic impact. However, jet aircraft cruising in the Stratosphere do create an impact from their contrails, although the extent of the damage to the environment is as yet unknown. Contrails can also trigger the formation of high-altitude Cirrus cloud thus creating a greater climatic effect. A 2015 study found that artificial cloudiness caused by contrail \"\"outbreaks\"\" reduce the difference between daytime and nighttime temperatures. The former are decreased and the latter are increased, in comparison\"",
        "passages": [
            "\"Aircraft flying at high altitude form condensation trails or contrails in the exhaust plume of their engines. While in the Troposphere these have very little climatic impact. However, jet aircraft cruising in the Stratosphere do create an impact from their contrails, although 